## Домашнее задание от 24.10.2024

##### Ваша задача: предсказать клиентов, которые намереваются уйти (churn). Необходимо продемонстрировать этапы подготовки данных:
##### Корректность загрузки
##### Проверка на пропуски
##### Проверка на аномалии
##### Группировки данных по категориям в зависимости от целевой переменной (выводы о данных, например, отток по типу карты)
##### Визуализации: гистограммы, тепловые карты, pairplot-ы, violin
##### Кодирование данных (через one-hot и через уникальное значение)
### Признаки и их значение 
###### churn - ушедший (Attrited) или текущий (Existing) клиент
###### Customer_Age - Демографическая переменная-возраст клиента в годах
###### Gender - Демографические переменные - M=мужской, F=женский
###### Dependent_count - Демографическая переменная-число иждивенцев
###### Education_Level - Демографическая переменная - образовательная квалификация владельца счета (пример: средняя школа, выпускник колледжа и т. д.)
###### Marital_Status - Демографическая переменная-женат, холост, разведен, неизвестен
###### Income_Category - Демографическая переменная - категория годового дохода владельца счета (< $40K, $40K - 60K, $60K - $80K, $80K-$120K, > $120K, неизвестно)
###### Card_Category - Переменная продукта-Тип карты (Синяя, серебряная, золотая, Платиновая)
###### Months_on_book - Период взаимоотношений с банком
###### Total_Relationship_Count - Общее количество продуктов, находящихся у клиента
###### Months_Inactive_12_mon - Количество месяцев бездействия за последние 12 месяцев
###### Contacts_Count_12_mon - Количество контактов за последние 12 месяцев
###### Credit_Limit - Кредитный лимит по кредитной карте
###### Total_Revolving_Bal - Общий оборотный остаток на кредитной карте
###### Avg_Open_To_Buy - Открытая для покупки кредитная линия (в среднем за последние 12 месяцев)
###### Total_Amt_Chng_Q4_Q1 - Изменение суммы сделки (4 квартал по сравнению с 1 кварталом)
###### Total_Trans_Amt - Общая сумма сделки (за последние 12 месяцев)
###### Total_Trans_Ct - Общее количество транзакций (за последние 12 месяцев)
###### Total_Ct_Chng_Q4_Q1 - Изменение количества транзакций (Q4 по сравнению с Q1)
###### Avg_Utilization_Ratio - Средний Коэффициент Использования Карт

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

seed = 24

ModuleNotFoundError: No module named 'sklean'

### Выводим верхние и нижние элементы таблицы

In [ ]:
df = pd.read_csv('churn.csv')
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()
df.shape

In [ ]:
df.describe()

### Смотрим дупликаты и null

In [ ]:
num_duplicated = df.duplicated().sum()
f'{num_duplicated} дупликатов найдено'

In [ ]:
sns.heatmap(df.isnull(), cmap='YlOrBr', vmin=0, vmax=1)

### Дубликатов и пустых значений нет, значит, смотрим выбросы

In [ ]:
df_select_out = df[df.columns[11:]]
outlier_cols = []

for column in df_select_out.columns:
    # Calculate the IQR (Interquartile Range)
    Q1 = df_select_out[column].quantile(0.25)
    Q3 = df_select_out[column].quantile(0.75)
    IQR = Q3 - Q1

    # Identify outliers based on the IQR
    outliers = (df_select_out[column] < Q1 - 1.5 * IQR) | (df_select_out[column] > Q3 + 1.5 * IQR)

    # Check if there are any outliers in the column
    if any(outliers):
        outlier_cols.append(column)

# Print columns with outliers
print("Columns with outliers:", outlier_cols)

### Больше всего выбросов в Credit_Limit, Avg_Open_To_Buy, Total_Amt_Chng_Q4_Q1

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=len(df_select_out.columns), figsize=(20, 5))

# Create box plots for each column
for i, column in enumerate(df_select_out.columns):
    sns.boxplot(x=df_select_out[column], ax=axes[i], color='skyblue')
    axes[i].set_title(f'{column}')

plt.tight_layout()
plt.show()

### Обработка выбросов (просто удалю)

In [ ]:
def del_outliers(frame, column):
    Q1 = frame[column].quantile(0.25)
    Q3 = frame[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    return frame[(frame[column] >= lower_bound) & (frame[column] <= upper_bound)]

for col in outlier_cols:
    df = del_outliers(df, col)
df

In [ ]:
df['Customer_Age'].unique()
avg_revolving = df.groupby('Customer_Age')['Total_Revolving_Bal'].mean().reset_index()

### Просто график, решил попробовать вывести 

In [ ]:
sns.set(style="darkgrid")
plt.figure(figsize=(14, 6))
sns.barplot(x='Customer_Age', y='Total_Revolving_Bal', data=avg_revolving)
plt.title('Average Revolving Balance ')
plt.xlabel('Age')
plt.ylabel('Avg Revolving Balance')

plt.show()

In [ ]:
df['Card_Category'].value_counts()
aim = df['churn']

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='Card_Category', hue='churn', data=df, palette='rocket')
plt.ylabel('Churn')
plt.xlabel('Count')
plt.title('Распределение ушедших и оставшихся клиентов по категории карты')
plt.legend(title='Category')

### Очень мало человек осталось, соу почти ничего не видно в 3 последних столбцах
### Также сделаю несколько круговых диаграмм

In [ ]:
income_types = df['Income_Category'].value_counts()
# income_types - количество каждого unique значения
plt.figure(figsize=(8, 8))
plt.pie(income_types, labels=income_types.index, autopct='%1.1f%%', startangle=140, colors=plt.cm.Set3.colors)

plt.legend()
plt.title('% распределения людей по Income Category')
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
sns.violinplot(df_select_out, x=df['Income_Category'], y=df['Customer_Age'], palette='mako')
plt.xlabel('Income_Category')
plt.ylabel('Customer_Age')
plt.title('Распределение категории дохода в зависимости от возраста')
plt.show()

### Закодируем object в int

In [ ]:
categorial_columns = df.select_dtypes(include=['object']).columns.tolist()
encoder = OrdinalEncoder()
encoded_data = encoder.fit_transform(df[categorial_columns])
df[categorial_columns] = encoded_data.astype(int)
df

### Корреляция всех значений

In [ ]:
plt.figure(figsize=(15,0.5))
sns.heatmap(df.corr().loc[['churn']], annot=True, vmax=1, vmin=0, cmap=sns.color_palette('flare', as_cmap=True), fmt='.2f')

In [ ]:
plt.figure(figsize=(16, 12))
sns.heatmap(df.corr(), annot=True, vmax=1, vmin=0, cmap='flare', fmt='.2f')

##### Видим, что целевая переменная коррилерирует в основном:
##### Total_Trans_Ct - Общее количество транзакций (за последние 12 месяцев)¶
##### Total_Ct_Chng_Q4_Q1 - Изменение количества транзакций (Q4 по сравнению с Q1)
##### Total_Revolving_Bal - Общий оборотный остаток на кредитной карте
##### Total_Trans_Amt - Общая сумма сделки (за последние 12 месяцев)
##### Также можно рассмотреть цепную корреляцию по столбцам: 
##### Total_Revolving_Bal -> Avg_Utilization_Ratio -> churn
##### Total_Trans_Ct -> Total_Ct_Chng_Q4_Q1 -> churn

### Обучение модельки

In [ ]:
target = 'churn'
np.random.seed(seed)

test_size = 0.25
data_train, data_test, y_train, y_test = train_test_split(
    df[df.columns.drop('churn')],
    np.array(df['churn']),
    test_size = test_size,
    randim_state=seed
)
print(f'Train: {data_train.shape} {y_train.shape}')
print(f'Test: {data_test.shape} {y_test.shape}')